In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk", font_scale=1.4)
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

import random
import matplotlib.pyplot as plt
import copy






In [3]:
import requests
import json

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [5]:
import dill
import base64

In [6]:
path = 'XRayDataSet/'
#path = '/content/drive/MyDrive/Colab Notebooks/XRayDataSet/'
image_dataset = torchvision.datasets.ImageFolder(root=path)

In [7]:
train_dataset, test_dataset = train_test_split(image_dataset, test_size=0.2)


print(len(train_dataset))
print(len(test_dataset))

class_num_covid = image_dataset.class_to_idx['Covid-19']
class_num_not_covid = image_dataset.class_to_idx['ANo_findings']
print('Covid-19 Class: ', class_num_covid)
print('Not-Covid-19 Class: ', class_num_not_covid)

train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

test_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

# client_dataset = []
# for i in range(len(train_dataset)):
#     # client_dataset.append((train_transforms(train_dataset[i][0]), torch.Tensor([class_num_covid]) if train_dataset[i][1] == class_num_covid else torch.Tensor([class_num_not_covid])))
#     client_dataset.append((train_transforms(train_dataset[i][0]), torch.Tensor([1,0]) if train_dataset[i][1] == class_num_covid else torch.Tensor([0,1])))
#
# client_loader = torch.utils.data.DataLoader(dataset=client_dataset, batch_size=32)


504
126
Covid-19 Class:  1
Not-Covid-19 Class:  0


In [8]:
def get_global_weights_from_server():
  res = requests.get("http://127.0.0.1:7000/get_global_model/")
  jsn = json.loads(res.text)
  return jsn['global_model']


def send_local_gradients_to_server(local_grads):
  s_obj = dill.dumps({
    'local_grads': local_grads
  })

  encoded = base64.b64encode(s_obj)

  data_post = {"local_grads": encoded.decode('ascii')}
  data_post = json.dumps(data_post)

  res = requests.post("http://127.0.0.1:7000/collect_model_updates/", data=data_post)
  if res.status_code == 200:
    return True
  return False

In [9]:
def c_mat(y_true, y_pred):
    _t = [int(1 if i[0] < i[1] else 0) for i in y_true]
    # print('y_pred: ' + str(y_pred))
    # _p = [int(1 if i[0] > 0 else 0) for i in y_pred]
    _p = [int(1 if i[0] < i[1] else 0) for i in y_pred]
    # print('target: ' + str(_t))
    # print('predicted: ' + str(_p))
    tn, fp, fn, tp = confusion_matrix(_t, _p, labels=[0, 1]).ravel()
    # print('tn: ' + str(tn), ' fp: ' + str(fp), ' fn: ' + str(fn), ' tp: ' + str(tp))
    # print('fp: ' + str(fp))
    # print('fn: ' + str(fn))
    # print('tp: ' + str(tp))
    return tn, fp, fn, tp

class FL_Client():
    def __init__(self, input , target):
        self.input = input
        self.target = target
        self.ae = torch.load("model1.pt")
        # self.criterion = nn.MSELoss()
        # self.criterion = nn.BCEWithLogitsLoss()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.ae.parameters(), lr = 2e-5)

    def set_input_and_target(self, input , target):
        self.input = input
        self.target = target

    def train_model(self):
      self.optimizer.zero_grad()
      #print(self.input.shape)
      output = self.ae(self.input)
      self.target.requires_grad = False
      #print("Output: ", output)
      tn, fp, fn, tp = c_mat(self.target, output)
      loss = self.criterion(output, self.target)
      loss.backward()
      return loss.item(), tn, fp, fn, tp

    def get_gradients(self):
      with torch.no_grad():
        gradients_arr = []
        for param in self.ae.parameters():

          #if param.grad is not None:
          if param.requires_grad==True:
            grad = param.grad.numpy()
            gradients_arr.append(grad)

          #gradients_arr.append(grad)
          #gradients_arr.append(param.grad.numpy())

        return gradients_arr

    def set_weights(self, weights):
      with torch.no_grad():
        i = 0
        for name, param in self.ae.named_parameters():
          param.data = torch.Tensor(weights[i])
          i += 1
    def train_test_model_and_get_loss(self, input, target):
      with torch.no_grad():
        output = self.ae(input)
        tn, fp, fn, tp = c_mat(target, output)
        loss = self.criterion(output, target)
        return loss.item(), tn, fp, fn, tp


In [9]:
train_dataset1, test_dataset1 = train_test_split(image_dataset, test_size=0.2, random_state=40)
client_dataset1 = []
for i in range(len(train_dataset1)):
    # client_dataset.append((train_transforms(train_dataset[i][0]), torch.Tensor([class_num_covid]) if train_dataset[i][1] == class_num_covid else torch.Tensor([class_num_not_covid])))
    client_dataset1.append((train_transforms(train_dataset1[i][0]), torch.Tensor([1,0]) if train_dataset1[i][1] == class_num_covid else torch.Tensor([0,1])))

client_loader1 = torch.utils.data.DataLoader(dataset=client_dataset1, batch_size=128)

In [10]:
clients_arr = []
for batch_idx, (train_features, train_labels) in enumerate(client_loader1):
    clients_arr.append(FL_Client(train_features, train_labels))
length=len(clients_arr)

In [13]:
# NUM OF CLIENTS = TOTAL IMAGES / BATCH SIZE
fl_iter = 10
threshold = int(len(train_dataset1) / 10)
train_loss_arr1 = []
train_acc_arr1 = []
train_prec_arr1 = []
train_recall_arr1 = []
train_f1_arr1 = []

print("FL_ITER: ", fl_iter)
print("NUM_CLIENTS_PER_FL_ITER: ", threshold)
print("Batch Idx Size: ", len(client_loader1))


for epoch in range(1, fl_iter + 1):
  global_weights = get_global_weights_from_server()
  local_gradients = []
  net_loss = 0
  net_i = 0
  tn = 0
  fp = 0
  fn = 0
  tp = 0
  count=0
  _accuracy=0
  _precision=0
  _recall=0
  _f1=0

  for fl_client in clients_arr:

    copy_fl_client = copy.deepcopy(fl_client)
    copy_fl_client.set_weights(copy.deepcopy(global_weights))
    local_loss, _tn, _fp, _fn, _tp = copy_fl_client.train_model()
    net_loss += local_loss
    net_i += 1
    tn += _tn
    fp += _fp
    fn += _fn
    tp += _tp
    _accuracy += (_tp+_tn)/(_tp+_fp+_tn+_fn)
    _precision += _tp/(_tp+_fp)
    _recall += _tp/(_tp+_fn)
    _f1 += 2/((1/_recall) + (1/_precision))
    count+=1
    #local_gradients.append(fl_client.get_gradients())
    print("--------Epoch ",epoch,"Client ", net_i,". Local Loss: ", local_loss, ". L-AC: ", _accuracy, ". L-P: ", _precision, ". L-R: ", _recall, ". L-F1: ", _f1)

    if send_local_gradients_to_server(copy_fl_client.get_gradients()) is not True:
            print("Error sending weights to server")
            break
    del copy_fl_client



  loss = net_loss / net_i
  train_loss_arr1.append(loss)
  #accuracy = (tp+tn)/(tp+fp+tn+fn)
  #precision = tp/(tp+fp)
  #recall = tp/(tp+fn)
  #f1 = 2/((1/recall) + (1/precision))
  accuracy=_accuracy/count
  precision=_precision/count
  recall=_recall/count
  f1=2/((1/recall) + (1/precision))
  train_acc_arr1.append(accuracy)
  train_prec_arr1.append(precision)
  train_recall_arr1.append(recall)
  train_f1_arr1.append(f1)

  print("Epoch ", epoch, ". Loss: ", loss, ". Accuracy: ", accuracy, ". Precision: ", precision, ". Recall: ", recall, ". F1-Score: ", f1)

FL_ITER:  10
NUM_CLIENTS_PER_FL_ITER:  50
Batch Idx Size:  4
--------Epoch  1 Client  1 . Local Loss:  0.6856427788734436 . L-AC:  0.5703125 . L-P:  0.835820895522388 . L-R:  0.56 . L-F1:  0.6706586826347306
--------Epoch  1 Client  2 . Local Loss:  0.664919376373291 . L-AC:  1.171875 . L-P:  1.692963752665245 . L-R:  1.1540594059405942 . L-F1:  2.0431664803138756
--------Epoch  1 Client  3 . Local Loss:  0.6836732625961304 . L-AC:  1.7265625 . L-P:  2.589515476803176 . L-R:  1.658913774872633 . L-F1:  4.065458786591625
--------Epoch  1 Client  4 . Local Loss:  0.684779167175293 . L-AC:  2.2682291666666665 . L-P:  3.4076972949849944 . L-R:  2.158913774872633 . L-F1:  6.708691639907341
Epoch  1 . Loss:  0.6797536462545395 . Accuracy:  0.5670572916666666 . Precision:  0.8519243237462486 . Recall:  0.5397284437181582 . F1-Score:  0.660808213328929
--------Epoch  2 Client  1 . Local Loss:  0.5043511390686035 . L-AC:  0.78125 . L-P:  0.78125 . L-R:  1.0 . L-F1:  0.8771929824561403
--------E

TEST SET

In [10]:
test_client_dataset = []
for i in range(len(test_dataset)):
    test_client_dataset.append((train_transforms(test_dataset[i][0]), torch.Tensor([1,0]) if test_dataset[i][1] == class_num_covid else torch.Tensor([0,1])))
test_client_loader = torch.utils.data.DataLoader(dataset=test_client_dataset, batch_size=32)
len(test_client_loader)

4

In [14]:
test_clients_arr = []
for batch_idx, (test_features, test_labels) in enumerate(test_client_loader):
  test_clients_arr.append(FL_Client(test_features, test_labels))

In [27]:
test_loss_arr = []
test_acc_arr = []
test_prec_arr = []
test_recall_arr = []
test_f1_arr = []

_accuracy=0
_precision=0
_recall=0
_f1=0
global_weights = get_global_weights_from_server()

for batch_idx, (test_features, test_labels) in enumerate(test_client_loader):

    test_fl_client = FL_Client(test_features, test_labels)
    test_fl_client.set_weights(copy.deepcopy(global_weights))
    test_clients_arr.append(test_fl_client)
    local_loss, _tn, _fp, _fn, _tp = test_fl_client.train_test_model_and_get_loss(test_features,test_labels)
    _accuracy += (_tp+_tn)/(_tp+_fp+_tn+_fn)
    _precision += _tp/(_tp+_fp)
    _recall += _tp/(_tp+_fn)
    _f1 = 2/((1/_recall) + (1/_precision))
    test_loss_arr.append(local_loss)
    test_acc_arr.append(_accuracy)
    test_prec_arr.append(_precision)
    test_recall_arr.append(_recall)
    test_f1_arr.append(_f1)

    print("Batch ", batch_idx, ". Loss: ", local_loss, ". Accuracy: ", _accuracy, ". Precision: ", _precision, ". Recall: ", _recall, ". F1-Score: ", _f1)

Batch  0 . Loss:  0.2877863049507141 . Accuracy:  0.9375 . Precision:  0.9230769230769231 . Recall:  1.0 . F1-Score:  0.9600000000000002
Batch  1 . Loss:  0.2274615317583084 . Accuracy:  1.875 . Precision:  1.8461538461538463 . Recall:  2.0 . F1-Score:  1.9200000000000004
Batch  2 . Loss:  0.2729840874671936 . Accuracy:  2.84375 . Precision:  2.809116809116809 . Recall:  3.0 . F1-Score:  2.901422265816577
Batch  3 . Loss:  0.2654171884059906 . Accuracy:  3.810416666666667 . Precision:  3.769116809116809 . Recall:  4.0 . F1-Score:  3.8811277026432367


In [28]:

test_loss = np.sum(test_loss_arr)/ len(test_loss_arr)
test_accuracy = _accuracy/ len(test_acc_arr)
test_precision = _precision/ len(test_prec_arr)
test_recall= _recall/len(test_recall_arr)
test_f1= _f1/ len(test_f1_arr)


print("Batch ", batch_idx, ". Loss: ", test_loss, ". Accuracy: ", test_accuracy, ". Precision: ", test_precision, ". Recall: ", test_recall, ". F1-Score: ", test_f1)


Batch  3 . Loss:  0.2634122781455517 . Accuracy:  0.9526041666666667 . Precision:  0.9422792022792023 . Recall:  1.0 . F1-Score:  0.9702819256608092


In [1]:
plt.figure(figsize=(8,5))
plt.plot(range(len(train_loss_arr1)),train_loss_arr1,color='dodgerblue')
plt.title('Model Convergence ')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.tight_layout()

NameError: name 'plt' is not defined

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(len(train_acc_arr1)),train_acc_arr1,color='dodgerblue')
plt.title('Accuracy Convergence')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(len(train_prec_arr1)),train_prec_arr1,color='dodgerblue')
plt.title('Precision Convergence')
plt.ylabel('precision')
plt.xlabel('epochs')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(len(train_recall_arr1)),train_recall_arr1,color='dodgerblue')
plt.title('Recall Convergence')
plt.ylabel('recall')
plt.xlabel('epochs')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(range(len(train_f1_arr1)),train_f1_arr1,color='dodgerblue')
plt.title('F1 Convergence ')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.tight_layout()